In [67]:
#%pip install torch --user
import json
import os
import numpy as np 
import pandas as pd
import torch
import torch.functional as F
import torch.nn as nn

%pip install transformers --user
%pip install openpyxl --user

from transformers import BertTokenizerFast,  BatchEncoding, BertModel, \
                            BertForTokenClassification
from tokenizers import Encoding

# %pip install ipywidgets --upgrade
# %pip install jupyter --upgrade
# %pip install seaborn --user
# %pip install sentencepiece

import seaborn as sns 
import matplotlib.pyplot as plt

import tqdm


torch.manual_seed(42)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


In [68]:
import pandas as pd
from utils import preproc
df = pd.read_excel("data/appliedAI_Use Case Library - Risk Class_TUM.ai.xlsx", index_col=0, header=2)

df = preproc(df)






In [69]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df, df['risk'],
                                                    stratify=df['risk'], 
                                                    test_size=0.25,
                                                   random_state=42)

In [70]:
y_test

Use Case ID
86      low-risk
57      low-risk
8       low-risk
30      low-risk
114     low-risk
36     high-risk
56     high-risk
87      low-risk
62      low-risk
92      low-risk
91      low-risk
80      low-risk
108     low-risk
89      low-risk
7       low-risk
5      high-risk
68      low-risk
85      low-risk
32      low-risk
42     high-risk
21      low-risk
76      low-risk
49      low-risk
34      low-risk
117     low-risk
Name: risk, dtype: object

In [71]:
from torch.utils.data import Dataset


# create a dataset - currently all files loaded beforehand (woeks for smaller datasets)
class RDataset(Dataset):
    # A pytorch dataset class for holding data for a text labeling task.
    # gets a parent directory of several datasets' directories
    def __init__(self, model_name, X, y):
        '''
        Takes as input the name of a file containing sentences with a classification label (comma separated) in each line.
        Stores the text data in a member variable X and labels in y
        '''
        
        # Load a pre-trained tokenizer
        self.tokenizer = AutoTokenizer.from_pretrained(model_name) 

        self.encodings = self.tokenizer(X["desc"].to_list(), return_tensors="pt", padding=True)
        self.y = y.replace({"low-risk": 0.0, "high-risk": 1.0}).to_list()
        
        

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.y[idx])
        return item

    def __len__(self):
        return len(self.y)


In [72]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

model_name = "MoritzLaurer/DeBERTa-v3-xsmall-mnli-fever-anli-ling-binary"
dataset_train = RDataset(model_name,  X_train, y_train)
dataset_test = RDataset(model_name, X_test, y_test)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=1, ignore_mismatched_sizes=True)
# model.config.num_labels = 1


/storage/brno2/home/xstary1/brno6/.local-adv.SIF/lib/python3.8/site-packages/transformers/convert_slow_tokenizer.py:446: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at MoritzLaurer/DeBERTa-v3-xsmall-mnli-fever-anli-ling-binary and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([2, 384]) in the checkpoint and torch.Size([1, 384]) in the model instantiated
- classifier.bias: found shape torch.Size([2]) in the checkpoint and torch.Size([1]) in the model instantiated
You should 

In [73]:
from torch.utils.data import DataLoader



In [74]:
# Check PyTorch GPU capabilities:

print("\nPyTorch:")
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('%d GPU(s) available.' % torch.cuda.device_count())
    print('GPU-Name:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

# device = torch.device("cpu")

model = model.to(device)


PyTorch:
1 GPU(s) available.
GPU-Name: NVIDIA A100-SXM4-40GB


In [75]:
#
## other training parameters
clip = 0.25            #gradient clipping
lr = 0.00001#0.00003           #initial learning rate
wdecay=1.2e-6          #weight decay applied to all weights
epochs = 2             #maximum number of epochs
#save = 'models/model.pt'      #path to save the final model

train_max_number_batches = -1 # only for the sake of debugging. Set to -1 to be ignored
inference_max_number_batches = -1 # only for the sake of debugging. Set to -1 to be ignored

## log parameters
log_interval = 100     #log interval during training
log_interval_val = 100 #log interval during validation

In [90]:

class Learner():
    def __init__(self,train_loader, val_loader, model, optimizer, device, loss_fn=None):
        self.train_loader= train_loader
        self.val_loader = val_loader
        self.model = model 
        self.optimizer = optimizer
        self.loss_fn = loss_fn
        self.device = device
        
    def evaluate(self, loader):
        eval_loss = 0
        self.model.eval()
        for b, inp in tqdm.tqdm(enumerate(loader), total=len(loader), position=0, leave=True):
            with torch.no_grad():
                inp = {k:v.clone().detach().to(device) for (k,v) in inp.items()}
                
                output = self.model(**inp)
                #loss = self.loss_fn(log_pred, y)
                logits = output.logits
                
                loss = self.loss_fn(logits, inp["labels"], pos_weight=torch.tensor(9))
                
                eval_loss += loss.item()
            
        eval_loss /= len(loader)
        
        return eval_loss
    
    def train(self,n_epochs, train_losses, val_losses, early_stopping=True, es_limit=3):

        if early_stopping:
            min_val_loss = np.inf
            no_improve = 0
        
        
        for e in range(n_epochs):
            # train
            train_loss = 0
            self.model.train()
            for b, inp in tqdm.tqdm(enumerate(self.train_loader), total=len(self.train_loader), position=0, leave=True):
                inp = {k:v.clone().detach().to(device) for (k,v) in inp.items()}

                self.optimizer.zero_grad()
                output = self.model(**inp)

                logits = output.logits

                print(inp["labels"])
                loss = self.loss_fn(logits, inp["labels"], pos_weight=torch.tensor(10))

                loss.backward()

                self.optimizer.step()

            # evaluate (train, validation)
            train_loss = self.evaluate(self.train_loader)
            val_loss = self.evaluate(self.val_loader)

            # early stopping
            if early_stopping:
                if val_loss < min_val_loss:
                    min_val_loss = val_loss
                    torch.save(self.model, "models/deberta")
                    no_improve = 0
                else:
                    no_improve += 1
            if no_improve > es_limit:
                print("Early stopped")
                self.model = torch.load("models/deberta")
                break
            
            print(f"After {e+1} epochs: ")
            print(f"Train loss: {train_loss:.3}")
            print(f"Val loss: {val_loss:.3}\n")
                
            train_losses.append(train_loss)
            val_losses.append(val_loss)
        return train_losses, val_losses
    
    

train_loader = DataLoader(dataset_train, batch_size=8, shuffle=True, num_workers=0)
val_loader = DataLoader(dataset_test, batch_size=16, shuffle=True, num_workers=0)

optim = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=wdecay)
loss_fn = torch.nn.functional.binary_cross_entropy_with_logits

train_losses, val_losses = [], []

In [91]:
learner = Learner(train_loader, val_loader, model, optim, device, loss_fn=loss_fn)
train_losses, val_losses = learner.train(50, train_losses, val_losses)

  0%|          | 0/10 [00:00<?, ?it/s]/var/tmp/pbs.12802678.meta-pbs.metacentrum.cz/ipykernel_894246/1032613092.py:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
/storage/brno2/home/xstary1/brno6/.local-adv.SIF/lib/python3.8/site-packages/transformers/models/deberta_v2/modeling_deberta_v2.py:745: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_scores = torch.bmm(query_layer, key_layer.transpose(-1, -2)) / torch.tensor(
/storage/brno2/home/xstary1/brno6/.local-adv.SIF/lib/python3.8/site-packages/transformers/models/deberta_v2/modeling_deberta_v2.py:829: UserWarning: To copy construct from a tensor, it i

tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')


 30%|███       | 3/10 [00:00<00:00,  8.70it/s]

tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([1., 0., 0., 1., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 1., 1., 0., 1., 0., 0., 0.], device='cuda:0')


 70%|███████   | 7/10 [00:00<00:00,  9.39it/s]

...

tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 1., 0., 0., 1., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')


  0%|          | 0/10 [00:00<?, ?it/s]

tensor([0., 1., 1., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0.], device='cuda:0')


100%|██████████| 2/2 [00:00<00:00, 53.15it/s]


Early stopped


In [92]:
def predict(model, loader, device="cpu", verbose=False):
    model.eval()
    model.to(device)
    
    y_pred = []
    y_true = []
    
    for b, inp in tqdm.tqdm(enumerate(loader), total=len(loader), position=0, leave=True):
        with torch.no_grad():
            inp = {k:v.to(device) for (k,v) in inp.items()}    
            output = model(**inp)
            
        y_true.append(inp["labels"])
        print(inp["labels"])
        #print(amask)
        print(output.logits)
        y_pred.append((output.logits > 0).float())
    #print(len(y_pred))
    #print(y_pred[0].shape)
    return torch.cat(y_pred, dim=0).to("cpu"), torch.cat(y_true, dim=0).to("cpu") 

pred_train, y_train = predict(model, train_loader, device=device, verbose=False)
pred_val, y_val = predict(model, val_loader, device=device, verbose=False)
# pred_test, y_test = predict(model, test_loader, device=device, verbose=False)

  0%|          | 0/10 [00:00<?, ?it/s]/var/tmp/pbs.12802678.meta-pbs.metacentrum.cz/ipykernel_894246/1032613092.py:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
  0%|          | 0/2 [00:00<?, ?it/s]

tensor([0., 0., 0., 1., 1., 0., 0., 1.], device='cuda:0')
tensor([-2.5821, -2.6358, -2.6894,  2.2186,  2.1958, -2.6956, -2.7522,  2.2867],
       device='cuda:0')
tensor([1., 0., 0., 0., 0., 1., 0., 0.], device='cuda:0')
tensor([ 2.1468, -2.6989, -2.6235, -2.6923, -2.6467,  2.2090, -2.6235, -2.6923],
       device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([-2.7415, -2.4818, -2.6506, -2.6419, -2.7578, -2.6593, -2.5627, -2.7123],
       device='cuda:0')
tensor([0., 1., 0., 0., 0., 1., 0., 0.], device='cuda:0')
tensor([ 2.2090,  2.1793, -2.5806, -2.4287, -2.5437,  2.2670, -2.7243, -2.7422],
       device='cuda:0')
tensor([0., 0., 0., 0., 1., 0., 0., 0.], device='cuda:0')
tensor([-2.4709, -2.6875, -2.6983, -2.7205,  2.2444, -2.7782, -2.6785, -2.5667],
       device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([-2.5133, -2.6366, -2.5085, -2.5839, -2.7569, -2.4353, -2.7698, -2.6358],
       device='cuda:0')
tensor([0., 0., 0., 0.

100%|██████████| 2/2 [00:00<00:00, 45.78it/s]

tensor([1., 0., 0., 0., 0., 0., 1., 0., 0.], device='cuda:0')
tensor([-2.6222, -2.7232, -2.7422, -2.5621, -2.5753, -0.9221, -1.6190,  2.2090,
        -2.6923], device='cuda:0')


In [93]:
y_train, y_test



(tensor([0., 0., 0., 1., 1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
         1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
         0.]),
 Use Case ID
 86      low-risk
 57      low-risk
 8       low-risk
 30      low-risk
 114     low-risk
 36     high-risk
 56     high-risk
 87      low-risk
 62      low-risk
 92      low-risk
 91      low-risk
 80      low-risk
 108     low-risk
 89      low-risk
 7       low-risk
 5      high-risk
 68      low-risk
 85      low-risk
 32      low-risk
 42     high-risk
 21      low-risk
 76      low-risk
 49      low-risk
 34      low-risk
 117     low-risk
 Name: risk, dtype: object)

In [94]:
pred_train

tensor([0., 0., 0., 1., 1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0.,
        0.])

In [95]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [96]:
for name, pred, y in zip(["TRAIN", "VAL"], [pred_train, pred_val], [y_train, y_val]):
    print(f"Summary on {name} set:")
    print()
    print(classification_report(y.ravel(), pred.ravel(), target_names=["high-res", "low-res"]))
    print(confusion_matrix(y.ravel(), pred.ravel()))
    print("*"*53)
    print()

Summary on TRAIN set:

              precision    recall  f1-score   support

    high-res       1.00      0.97      0.98        63
     low-res       0.83      1.00      0.91        10

    accuracy                           0.97        73
   macro avg       0.92      0.98      0.95        73
weighted avg       0.98      0.97      0.97        73

[[61  2]
 [ 0 10]]
*****************************************************

Summary on VAL set:

              precision    recall  f1-score   support

    high-res       0.87      0.95      0.91        21
     low-res       0.50      0.25      0.33         4

    accuracy                           0.84        25
   macro avg       0.68      0.60      0.62        25
weighted avg       0.81      0.84      0.82        25

[[20  1]
 [ 3  1]]
*****************************************************

